In [74]:
# Imports
import pandas as pd
import requests
import numpy as np
import matplotlib as plt
from bs4 import BeautifulSoup 
#from tabulate import tabulate

In [2]:
url = requests.get("https://www.fundsexplorer.com.br/ranking")

In [3]:
fii_bs4 = BeautifulSoup(url.text, "html.parser")

In [4]:
fii_table = fii_bs4.findAll(attrs = {'id' : 'scroll-wrapper'})
table = fii_table[0].findAll('table')

In [5]:
fii_df = pd.read_html(str(fii_table[0]))[0]

In [ ]:
fii_df

In [ ]:
fii_df.columns.values

In [8]:
fii_df.columns = ['ticker', 'Setor', 'price', 'liquidez',
       'Dividendo', 'DividendYield', 'DY (3M)Acumulado',
       'DY (6M)Acumulado', 'dy', 'DY (3M)Média',
       'DY (6M)Média', 'Dividenmediio', 'DY Ano', 'Variação Preço',
       'Rentab.Período', 'Rentab.Acumulada', 'PatrimônioLíq.', 'VPA',
       'pvpa', 'DYPatrimonial', 'VariaçãoPatrimonial',
       'Rentab. Patr.no Período', 'Rentab. Patr.Acumulada',
       'VacânciaFísica', 'VacânciaFinanceira', 'QuantidadeAtivos']

In [ ]:
fii_df.columns.values

In [10]:
fii_back = fii_df.copy()

In [11]:
test = fii_df[['ticker', 'price', 'dy', 'pvpa','liquidez']]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [13]:
test = test[test['price'].notna()]
test = test[test['dy'].notna()]

In [ ]:
print("--> DATA:")
display(test.isnull().sum())

In [16]:
test = test.sort_values(['dy'], ascending=False)

In [ ]:
test

In [18]:
unit = 'R$'
test['price'] = [sub.replace(unit, "").strip() for sub in test['price']]

In [ ]:
test

In [ ]:
test.dtypes

In [ ]:
test['price'] = test['price'].str.replace(',', '.')
test['price'] = pd.to_numeric(test['price'], errors='coerce')
test

In [22]:
test = test[test['price'].notna()]

In [ ]:
test['lid2'] = test['price'] * test['liquidez']
test

In [ ]:
test = test.sort_values(['lid2'], ascending=False)

In [34]:
test.reset_index(inplace=True, drop=True)

In [ ]:
indexNames = test[ (test['lid2'] <= 200000)].index
test.drop(indexNames , inplace=True)
test

In [53]:
ts = test.shape
ts = ts[0]

In [58]:
test = test.sort_values(['dy'], ascending=False)
test.reset_index(inplace=True, drop=True)

In [59]:
b1 = [*range(0, ts, 1)]

In [60]:
test['b1'] = pd.Series(b1)

In [ ]:
test

In [62]:
test = test.sort_values(['pvpa'], ascending=True)

In [63]:
test.reset_index(inplace=True, drop=True)

In [64]:
b2 = [*range(0, ts, 1)]
test['b2'] = pd.Series(b2)

In [ ]:
test

In [66]:
test = test.eval('Rank = b1 + b2')

In [ ]:
test = test.sort_values(['Rank'], ascending=True)

In [71]:
test.reset_index(inplace=True, drop=True)

In [ ]:
indexNames = test[(test.index > 10)].index
test.drop(indexNames , inplace=True)